In [1]:
import pandas as pd
import numpy as np
import requests
from pprint import pprint
from bs4 import BeautifulSoup
import datetime as dt

from api_keys import api_key

In [2]:
def request_song_info(artist_id):
    base_url = f'https://api.genius.com'
    headers = {f'Authorization': f'Bearer {api_key}'}
    search_url = f'{base_url}/artists/{artist_id}/songs'
    response = requests.get(search_url, headers=headers)

    return response

In [3]:
artist_id = 2
jay_z_dict = {}

response = request_song_info(artist_id)
json = response.json()
remote_song_info = None

for hit in json['response']['songs']:
    if str(artist_id) in str(hit['primary_artist']['id']):
        remote_song_info = hit
        print(hit['title'].lower())
        if remote_song_info:
            song_url = remote_song_info['url']
            jay_z_dict.update({hit['title'].lower() : song_url})
    if len(jay_z_dict) == 25:
        break

'03 bonnie & clyde
100$ bill
11 minute freestyle on hot 97
1-900-hustler
1999 westwood freestyle
2000 tim westwood freestyle
2002 funk flex freestyle
20 bag shorty
20 questions for jay-z
22 days challenge
22 two's
2 many hoes
30 something


In [4]:
# pprint(json)

In [5]:
print(jay_z_dict)
print(len(jay_z_dict))

{"'03 bonnie & clyde": 'https://genius.com/Jay-z-03-bonnie-and-clyde-lyrics', '100$ bill': 'https://genius.com/Jay-z-100-bill-lyrics', '11 minute freestyle on hot 97': 'https://genius.com/Cassidy-11-minute-freestyle-on-hot-97-lyrics', '1-900-hustler': 'https://genius.com/Jay-z-1-900-hustler-lyrics', '1999 westwood freestyle': 'https://genius.com/Jay-z-1999-westwood-freestyle-lyrics', '2000 tim westwood freestyle': 'https://genius.com/Jay-z-2000-tim-westwood-freestyle-lyrics', '2002 funk flex freestyle': 'https://genius.com/Jay-z-2002-funk-flex-freestyle-lyrics', '20 bag shorty': 'https://genius.com/Jay-z-20-bag-shorty-lyrics', '20 questions for jay-z': 'https://genius.com/John-paul-davis-20-questions-for-jay-z-annotated', '22 days challenge': 'https://genius.com/Jay-z-22-days-challenge-annotated', "22 two's": 'https://genius.com/Jay-z-22-twos-lyrics', '2 many hoes': 'https://genius.com/Jay-z-2-many-hoes-lyrics', '30 something': 'https://genius.com/Jay-z-30-something-lyrics'}
13


In [6]:
def scrap_song_url(url):
    page = requests.get(url)
    html = BeautifulSoup(page.text, 'html.parser')
    release_date = dt.datetime.strptime(html.find_all('span', {'class': 'metadata_unit-info metadata_unit-info--text_only'})[1].get_text(), '%B %d, %Y').strftime('%Y%m%d')
    lyrics = html.find('div', class_='lyrics').get_text()

    return release_date, lyrics

In [7]:
pprint(scrap_song_url(song_url))

('20061121',
 '\n'
 '\n'
 '[Intro]\n'
 "You ain't got enough stamps in ya passport\n"
 'To fuck with Young H-O (heheheh)\n'
 'International, uhh\n'
 'Show young boys how to do this thing\n'
 'The maturation of Jay-Zeezy, heh\n'
 'Check me out\n'
 '\n'
 '[Verse 1]\n'
 "30's the new 20 nigga I'm so hot still (uhh)\n"
 'Better broad, better automobile (uhh)\n'
 'Bet a yard (naw!) bet a hundred mil\n'
 "That by the song's end I'll probably start another trend\n"
 "I know everything you wan' do\n"
 'I did all that by the age of 21\n'
 "By 22, I had that brand new Ac' coupe\n"
 'I guess you could say that my legend just begun\n'
 "I'm, young enough to know the right car to buy\n"
 'Yet grown enough not to put rims on it\n'
 "I got that six-deuce with curtains so you can't see me\n"
 "And I didn't even have to put tints on it\n"
 "I don't got the bright watch, I got the right watch\n"
 "I don't buy out the bar, I bought the nightspot\n"
 'I got the right stock\n'
 "I, got, stockbrokers that's